In [1]:
:set -XOverloadedStrings

import Data.Maybe (fromJust)
import qualified Data.Set as S

import Prelude hiding ((^^))

import Duffer
import Duffer.Loose
import Duffer.Loose.Objects
import Duffer.WithRepo
import Duffer.Unified

duffer = withRepo "../.git"
resolveRef' = fmap fromJust . resolveRef
readObject' = fmap fromJust . readObject

Let's start with a raw representation of the most recent commit:

In [2]:
:!git show --format=raw -s

commit f816236922a2f70e092bace54ddf7c22796834e8
tree cae77a986bf4556f734dc3e35a1773013bc27b03
parent d3701eb2adf4c8133479fb633a47ec0ed81f97a5
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1487876264 -0500
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1487876264 -0500

    Reformat imports.

I'm currently on the `master` branch, so another way to get to this object is as follows:

In [3]:
duffer (resolveRef' "refs/heads/master")

tree cae77a986bf4556f734dc3e35a1773013bc27b03
parent d3701eb2adf4c8133479fb633a47ec0ed81f97a5
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1487876264 -0500
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1487876264 -0500

Reformat imports.

A commit refers to a `tree`, which is `git`'s way of storing a directory. An example tree looks like

<img src="https://git-scm.com/book/en/v2/book/10-git-internals/images/data-model-2.png">
Source: https://git-scm.com/book/en/v2/Git-Internals-Git-Objects

We can view the pretty-printed contents of a `git` object with `cat-file -p`. Each commit has a `tree` associated with it which represents a directory, in this case the root project folder.

In [4]:
:!git cat-file -p master^{tree}

040000 tree b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644 blob cf6f631cb9b448ae581e6cebb43fe0e1a4247918	.gitignore
100644 blob 910528adcdcd7127d8824061eb8c83471be6953e	.travis.yml
100644 blob 36f41422dbe820719bce44e0122259ad11580db0	HLint.hs
100644 blob 78614ecf3a7084b230af26d1badb7feae7a876d0	LICENSE
100644 blob 250804ca4920a0c38763f744e54f18703992eb12	README.md
100644 blob 9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644 blob 2e052aac04efc4037ecb79b7c073e638695546ca	default.nix
100644 blob 1165e2461ea3bfa3ed5f04cd9c54363116fe5ebd	duffer.cabal
040000 tree ac1d26d71059ac995f8df524159d0f2b93367f1c	notebooks
040000 tree 787554971c1c709fc4e6d143fdad13294387794b	presentation
100644 blob f54e9d41c4cd9ca748cdbe0eb4352c53cd5b93e3	release.nix
040000 tree fb45d36e7654860421ccdfd77309f7a68b13d48c	src
100644 blob 48ba3971983e95289ba37e449f1520f7ea1cb9d8	stack.yaml
040000 tree 24599c0302603b12dabf470da2519463c5b3d501	test

Again, we can obtain almost identical (modulo formatting) output with `duffer`:

In [5]:
duffer $ do
    master    <- resolveRef' "refs/heads/master"
    let tree  =  commitTreeRef master
    readObject' tree

040000	tree	b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644	blob	cf6f631cb9b448ae581e6cebb43fe0e1a4247918	.gitignore
100644	blob	910528adcdcd7127d8824061eb8c83471be6953e	.travis.yml
100644	blob	36f41422dbe820719bce44e0122259ad11580db0	HLint.hs
100644	blob	78614ecf3a7084b230af26d1badb7feae7a876d0	LICENSE
100644	blob	250804ca4920a0c38763f744e54f18703992eb12	README.md
100644	blob	9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644	blob	2e052aac04efc4037ecb79b7c073e638695546ca	default.nix
100644	blob	1165e2461ea3bfa3ed5f04cd9c54363116fe5ebd	duffer.cabal
040000	tree	ac1d26d71059ac995f8df524159d0f2b93367f1c	notebooks
040000	tree	787554971c1c709fc4e6d143fdad13294387794b	presentation
100644	blob	f54e9d41c4cd9ca748cdbe0eb4352c53cd5b93e3	release.nix
040000	tree	fb45d36e7654860421ccdfd77309f7a68b13d48c	src
100644	blob	48ba3971983e95289ba37e449f1520f7ea1cb9d8	stack.yaml
040000	tree	24599c0302603b12dabf470da2519463c5b3d501	test

`git` implements a giant hashtable on the filesystem using SHA1 as the hashing function. It stores all the past files and directory listings as `zlib`-compressed text files (with a header denoting object type and length) under `.git/objects` as follows:

1. Compute a SHA1 hash of the content.
2. `zlib`-compress the content.
3. Take the first 2 characters of the hash. This is the subdirectory under `.git/objects` where the content will be stored.
4. The remaining 38 characters of the hash are the filename.

<img src="//git-scm.com/book/en/v2/book/10-git-internals/images/data-model-3.png">
Source: https://git-scm.com/book/en/v2/Git-Internals-Git-Objects

For example, a decompressed commit looks like:

In [6]:
:!cat ../.git/objects/4b/d9b179bb166b85e3e889f9f263f1b5a26f3e34 | zlib-flate -uncompress

commit 287 tree 0b36647819f93c5523b5967d19cb131d88ab1be4
parent 2577894fb379a7cbe2e3bfd3ba325f4e451bbb5f
parent d72ae27a9ae58d49235ff9761cfae816b004d9b1
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400

Add porcelain.

In [7]:
duffer $ readObject' "4bd9b179bb166b85e3e889f9f263f1b5a26f3e34"

tree 0b36647819f93c5523b5967d19cb131d88ab1be4
parent 2577894fb379a7cbe2e3bfd3ba325f4e451bbb5f
parent d72ae27a9ae58d49235ff9761cfae816b004d9b1
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1473766850 -0400

Add porcelain.

In [8]:
:!git branch

  gh-pages
* master
  new-branch
  work-tree

In [9]:
duffer $ do
    current  <- resolveRef' "refs/heads/master"
    parent   <- fromJust <$> current ^^ 1
    fromJust <$> parent ~~ 1

tree 0dd9735585db9a630221fa775fa2a189288e1cc2
parent c95be33244a1a4f6d1e54f5dc34ad18ae709fef0
author Vaibhav Sagar <vaibhavsagar@gmail.com> 1487766127 -0500
committer Vaibhav Sagar <vaibhavsagar@gmail.com> 1487766127 -0500

Reorganise tests into loose and packed.

As mentioned previously, the hash of a `git` object uniquely identifies it in the giant hashtable that is `git`

In [10]:
tree <- duffer $ readObject' "a28aded05daa52ff5d0c77cd6186b1ce0faf7c8c"
hash tree

"a28aded05daa52ff5d0c77cd6186b1ce0faf7c8c"

`git` refers to files as `blob`s.

In [11]:
duffer $ readObject' "b75f4c9dbe3b61cacba052f23461834468832e41"

Copyright Vaibhav Sagar (c) 2015

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

    * Redistributions of source code must retain the above copyright
      notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
      copyright notice, this list of conditions and the following
      disclaimer in the documentation and/or other materials provided
      with the distribution.

    * Neither the name of Vaibhav Sagar nor the names of other
      contributors may be used to endorse or promote products derived
      from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DIS

The last type of `git` object is a `tag`, which gives a name to another `git` object.

In [12]:
duffer $ readObject' "d4b1e0343313ab60688cf0ddfa8ae5d8fe60ec23"

object 25354a5cfebca0261cdaa87ebef3a6b9dcb9c13a
type commit
tag test
tagger Vaibhav Sagar <vaibhavsagar@gmail.com> 1459935215 +1000

Test tag.

`duffer` is pretty great at reading `git` repositories, but that's not all you can do with it. You can also add content to a `git` repository with it:

In [13]:
import Data.ByteString.UTF8 (fromString, toString)
blob = Blob (fromString "hello world")
duffer $ writeLooseObject blob

"95d09f2b10159347eece71399a7e2e907ea3df4f"

In [14]:
:!git cat-file -p 95d09f2b10159347eece71399a7e2e907ea3df4f

hello world

In [15]:
:!git branch

  gh-pages
* master
  new-branch
  work-tree

In [16]:
currentCommit = resolveRef' "refs/heads/master"
duffer $ currentCommit >>= \commit -> updateRef "refs/heads/new-branch" commit

"f816236922a2f70e092bace54ddf7c22796834e8"

In [17]:
:!git branch

  gh-pages
* master
  new-branch
  work-tree

In [18]:
rootTree = duffer $ commitTreeRef <$> currentCommit >>= readObject'
rootTree

040000	tree	b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644	blob	cf6f631cb9b448ae581e6cebb43fe0e1a4247918	.gitignore
100644	blob	910528adcdcd7127d8824061eb8c83471be6953e	.travis.yml
100644	blob	36f41422dbe820719bce44e0122259ad11580db0	HLint.hs
100644	blob	78614ecf3a7084b230af26d1badb7feae7a876d0	LICENSE
100644	blob	250804ca4920a0c38763f744e54f18703992eb12	README.md
100644	blob	9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644	blob	2e052aac04efc4037ecb79b7c073e638695546ca	default.nix
100644	blob	1165e2461ea3bfa3ed5f04cd9c54363116fe5ebd	duffer.cabal
040000	tree	ac1d26d71059ac995f8df524159d0f2b93367f1c	notebooks
040000	tree	787554971c1c709fc4e6d143fdad13294387794b	presentation
100644	blob	f54e9d41c4cd9ca748cdbe0eb4352c53cd5b93e3	release.nix
040000	tree	fb45d36e7654860421ccdfd77309f7a68b13d48c	src
100644	blob	48ba3971983e95289ba37e449f1520f7ea1cb9d8	stack.yaml
040000	tree	24599c0302603b12dabf470da2519463c5b3d501	test

In [19]:
newFile = TreeEntry Regular "new-file" "95d09f2b10159347eece71399a7e2e907ea3df4f"
duffer $ do
    entries        <- liftIO $ treeEntries <$> rootTree
    let newEntries =  S.insert newFile entries
    newTree        <- writeLooseObject (Tree newEntries)
    let me         =  PersonTime "Vaibhav Sagar" "vaibhavsagar@gmail.com" "1461156164" "+1000" 
    let newCommit  =  Commit newTree ["d76238fed6c656183a4d4dcf287217a061043869"] me me "New commit."
    newHead        <- writeLooseObject newCommit
    updateRef "refs/heads/new-branch" newCommit

"c6781746a0da23f2431d5194c537626a49e35114"

In [20]:
newTree = duffer $ commitTreeRef <$> resolveRef' "refs/heads/new-branch" >>= readObject'
newTree

040000	tree	b8c0fd17ebd7bad61687fe06449b0a484b207efb	.ci
100644	blob	cf6f631cb9b448ae581e6cebb43fe0e1a4247918	.gitignore
100644	blob	910528adcdcd7127d8824061eb8c83471be6953e	.travis.yml
100644	blob	36f41422dbe820719bce44e0122259ad11580db0	HLint.hs
100644	blob	78614ecf3a7084b230af26d1badb7feae7a876d0	LICENSE
100644	blob	250804ca4920a0c38763f744e54f18703992eb12	README.md
100644	blob	9a994af677b0dfd41b4e3b76b3e7e604003d64e1	Setup.hs
100644	blob	2e052aac04efc4037ecb79b7c073e638695546ca	default.nix
100644	blob	1165e2461ea3bfa3ed5f04cd9c54363116fe5ebd	duffer.cabal
100644	blob	95d09f2b10159347eece71399a7e2e907ea3df4f	new-file
040000	tree	ac1d26d71059ac995f8df524159d0f2b93367f1c	notebooks
040000	tree	787554971c1c709fc4e6d143fdad13294387794b	presentation
100644	blob	f54e9d41c4cd9ca748cdbe0eb4352c53cd5b93e3	release.nix
040000	tree	fb45d36e7654860421ccdfd77309f7a68b13d48c	src
100644	blob	48ba3971983e95289ba37e449f1520f7ea1cb9d8	stack.yaml
040000	tree	24599c0302603b12dabf470da2519463c5b3d501	test